In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass(prompt='LangSmith API Key: ')

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt='OpenAI API Key: ')

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [3]:
# import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader, TextLoader, WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = DirectoryLoader("./Whatsapp_Chats", glob="*.txt", loader_cls=TextLoader)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [7]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [20]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context (which are past conversations between Sahil and his contacts on WhatsApp) to reply to the message at the end as Sahil.
Answer based on who is sending the message and Sahil's most recent chat history with that person.
The name of the person sending the message will be shown at the beginning of the message.
If you don't know the person, just say that you don't have their number saved, don't try to make up a random reply for a stranger.
Use three sentences maximum and keep the answer as close to how Sahil replies as possible.
Always use and emoji at the end of the answer.

{context}

Message: {messageIn}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "messageIn": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Shubham: Where are you bro?")

'Hey Shubham! All good. Just chilling in India. 😅👍'